In [46]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [47]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import avg, round, year
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [48]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [49]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')
df.createOrReplaceTempView('home_sales')



24/11/23 22:28:47 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


In [50]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?



# Extract the year from the `date` column and create a new column
df_with_year = df.withColumn("year", year("date"))

# Filter for four-bedroom houses
four_bedroom_houses = df_with_year.filter(df_with_year.bedrooms == 4)

# Calculate the average price per year
average_price_per_year = (
    four_bedroom_houses
    .groupBy("year")
    .agg(round(avg("price"), 2).alias("average_price"))
)

# Show the result
average_price_per_year.show()



+----+-------------+
|year|average_price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [51]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# Filter for homes with 3 bedrooms and 3 bathrooms
filtered_homes = df.filter((df.bedrooms == 3) & (df.bathrooms == 3))

# Calculate the average price for each year the home was built
average_price_per_year_built = (
    filtered_homes
    .groupBy("date_built")  # Assuming "date_built" is the column representing the year built
    .agg(round(avg("price"), 2).alias("average_price"))
)

# Show the result
average_price_per_year_built.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [52]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?


# Apply the filters
filtered_homes = df.filter(
    (df.bedrooms == 3) & 
    (df.bathrooms == 3) & 
    (df.floors == 2) & 
    (df.sqft_living >= 2000)
)

# Calculate the average price for each year the home was built
average_price_per_year_built = (
    filtered_homes
    .groupBy("date_built")  # Assuming "date_built" represents the year built
    .agg(round(avg("price"), 2).alias("average_price"))
)

# Show the result
average_price_per_year_built.show()



+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280317.58|
+----------+-------------+



In [53]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

#Start Timer
start_time = time.time()

# Calculate the average price per view rating
average_price_per_view = (
    df.groupBy("view")  # Group by the "view" column
    .agg(round(avg("price"), 2).alias("average_price"))  # Calculate average price and round
    .filter("average_price >= 350000")  # Filter for average price >= $350,000
    .orderBy("view", ascending=False)  # Order by descending view rating
)

# Show the results
average_price_per_view.show()

#End Timer
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.3800840377807617 seconds ---


In [54]:
# 7. Cache the the temporary table home_sales.
# Register the DataFrame as a temporary table or view
df.createOrReplaceTempView("home_sales")

# Cache the table
spark.catalog.cacheTable("home_sales")

In [55]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [56]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

#Start Timer
start_time = time.time()

# Run the query on the cached data
average_price_per_view_cached = (
    spark.sql("""
        SELECT 
            view, 
            ROUND(AVG(price), 2) AS average_price
        FROM home_sales
        GROUP BY view
        HAVING average_price >= 350000
        ORDER BY view DESC
    """)
)

# Show the results
average_price_per_view_cached.show()

#End Timer
print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9072549343109131 seconds ---


In [57]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
output_path = "path_to_output_directory/home_sales_partitioned"  # Replace with your desired output path

df.write.partitionBy("date_built").parquet(output_path, mode="overwrite")

print(f"Data successfully written to {output_path}, partitioned by 'date_built'.")

Data successfully written to path_to_output_directory/home_sales_partitioned, partitioned by 'date_built'.


In [58]:
# 11. Read the formatted parquet data.
# Specify the path to the Parquet data
parquet_path = "path_to_output_directory/home_sales_partitioned"  # Replace with the actual path

# Read the Parquet data
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows to verify the data
parquet_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [59]:
# 12. Create a temporary table for the parquet data.
# Register the Parquet DataFrame as a temporary table
parquet_df.createOrReplaceTempView("home_sales_parquet")

# Verify the table creation
spark.sql("SHOW TABLES").show()


+---------+------------------+-----------+
|namespace|         tableName|isTemporary|
+---------+------------------+-----------+
|         |        home_sales|       true|
|         |home_sales_parquet|       true|
+---------+------------------+-----------+



In [60]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

# Start Timer
start_time = time.time()

# Run the query on the Parquet data
average_price_per_view_parquet = (
    parquet_df.groupBy("view")  # Group by the "view" column
    .agg(round(avg("price"), 2).alias("average_price"))  # Calculate average price and round
    .filter("average_price >= 350000")  # Filter for average price >= $350,000
    .orderBy("view", ascending=False)  # Order by descending view rating
)

# Show the results
average_price_per_view_parquet.show()

# End Timer
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.8771059513092041 seconds ---


In [61]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [62]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? False
